In [18]:
import requests
import time, os
import random
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from IPython.core.display import display, HTML
from collections import defaultdict
import csv
import pandas as pd

In [329]:
'''
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver
'''

'\nfrom selenium import webdriver\nfrom selenium.webdriver.common.keys import Keys\nfrom selenium.webdriver.common.action_chains import ActionChains\n\nchromedriver = "/Applications/chromedriver" # path to the chromedriver executable\nos.environ["webdriver.chrome.driver"] = chromedriver\n'

In [138]:
base_url = 'https://www.pro-football-reference.com/players/'

#scrape Jared Goff first
#player = 'players/G/GoffJa00.htm'

In [194]:
def get_player_htm():
    '''
    Use name to get name for player's html page identifier
    return:
        dict of {first_part_htm_names : player_name}
    '''
    qb_list_file = './pfr_scraped/master_qb_list.csv'
    qb_df = pd.read_csv(qb_list_file, index_col=0)
    htm_name_dict = {}

    qb_df = qb_df['0'].str.split(expand=True).copy()
    qb_df[2] = qb_df[0] + ' ' + qb_df[1]
    qb_df.rename({0:'First', 1:'Last', 2:'Full'}, axis=1, inplace=True)
    qb_df['htm_name'] = qb_df['Last'].str.slice(stop=4) + qb_df['First'].str.slice(stop=2)
    htm_name_dict = dict(zip(qb_df['htm_name'], qb_df['Full']))
    return htm_name_dict

In [202]:
htm_name_dict = get_player_htm()
htm_name_dict
test_dict = {}
test_dict['MannPe'] = 'Peyton Manning'
test_dict

{'MannPe': 'Peyton Manning'}

In [251]:
len(htm_name_dict)

104

In [240]:
def scrape_page(htm_dict):
    '''
    Scrape page and store in a list of soup files
    arg:
        list of qb's name as html identifier
    return:
        list of players pages in soup form
    '''
    ua = UserAgent()
    user_agent = {'User-agent' : ua.random}
    player_page_soups = []
    error_list = []

    for qb_htm, qb_name in htm_dict.items():
        for x in range(6):
            name = qb_htm + '0' + str(x) + '.htm'
            add_url = qb_htm[0] + '/' + name
            response = requests.get(base_url + add_url, headers=user_agent)
            if response.status_code == 200:
                print(str(response.status_code) + add_url)
                page_soup = BeautifulSoup(response.text, 'lxml')
                page_name = page_soup.find('h1',{'itemprop':'name'}).text
                if page_name != qb_name:
                    time.sleep(random.random() + random.randint(1,4))
                    continue
                else:
                    break
            else:
                print('Something\'s wrong with:' + add_url)
                error_list.append(qb_name)
                break
        player_page_soups.append(page_soup)
        time.sleep(random.random() + random.randint(1,4))        
    return player_page_soups

In [254]:
def save_scrape_page(soup_list):
    qb_soup_list = [k.prettify() for k in soup_list]
    # add the custom word to standout, I chose BREAKHERE
    list_with_breaks = [m+'BREAKHERE' for m in qb_soup_list]
    compiled_list = "".join(list_with_breaks)
    with open('./pfr_scraped/qb_soup_list.txt', 'w') as file:
        file.write(compiled_list)

In [255]:
save_scrape_page(player_page_soups)

In [256]:
error_list

[]

In [30]:
def get_big_table(soup_list):
    # for each soup (aka players page), return big_table to get_stats and get_headers
    big_table = page_soup.find_all('table', {'id':'passing'})
    pass

In [71]:
def get_stats(table):
    stats = defaultdict(list)
    stat_table = table[0].find_all('tr', class_='full_table')
    for ele in stat_table:
        year = ele.find('a').text
        temp_stat = ele.find_all('td')
        for num in temp_stat:
            stats[year].append(num.text)
    
    return stats

In [72]:
stats = get_stats(big_table)

In [7]:
def get_headers(table):
    '''
    Grab stat headers from player's BS table
    arg:
        BS object containing players info (from scrape_page)
    return:
        stat labels in a list
    '''
    stat_header = []

    stat_table = table[0].find_all('tr')
    header_soup = stat_table[0].find_all('th')
    for header in header_soup:
        stat_header.append(header.text)
    return stat_header

In [9]:
headers = get_headers(big_table)

In [336]:
csv_file = 'Goff_test.csv'

with open(csv_file, 'w') as csvfile:
    csvfile.write(','.join(headers)+'\n')
    for key, value in stats.items():
        csvfile.write(key+','+','.join(value)+'\n')

In [337]:
pd.read_csv('Goff_test.csv')

,Year,Age,Tm,Pos,No.,G,GS,QBrec,Cmp,Att,...,Rate,QBR,Sk,Yds.1,NY/A,ANY/A,Sk%,4QC,GWD,AV
0,2016,22,LAR,qb,16,7,7,0-7-0,112,205,...,63.6,18.9,26,222,3.75,2.82,11.3,NaN,NaN,-2
1,2017,23,LAR,QB,16,15,15,11-4-0,296,477,...,100.5,52.1,25,172,7.24,7.72,5.0,1.0,1.0,15
2,2018,24,LAR,QB,16,16,16,13-3-0,364,561,...,101.1,63.1,33,223,7.52,7.69,5.6,4.0,4.0,18
3,2019,25,LAR,QB,16,16,16,9-7-0,394,626,...,86.5,NaN,22,170,6.90,6.46,3.4,1.0,2.0,12
